In [1]:
import matplotlib as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

# added
from folder import KFolder
import numpy as np
import os
from PIL import Image
from Augmentation import ToAugmantate
from shuffle import shuffle_data

In [2]:
def loss_plot(hist):
    train_epochs = list(range(1, len(hist.history['loss']) + 1))
    valid_epochs = list(range(1, len(hist.history['val_loss']) + 1))
    train_losses = hist.history['loss']
    valid_losses = hist.history['val_loss']

    train_min_loss = min(dict(zip(train_epochs, train_losses)).items(), key=lambda x: x[1])
    valid_min_loss = min(dict(zip(valid_epochs, valid_losses)).items(), key=lambda x: x[1])

    plt.plot(train_epochs, hist.history['loss'], linewidth=1, color='blue', label='Training Loss')
    plt.plot(train_min_loss[0], train_min_loss[1], color='black', marker='o',
             label=f'Training Best Epoch - {train_min_loss[0]}')
    plt.plot(valid_epochs, hist.history['val_loss'], linewidth=1, color='red', label='Validation Loss')
    plt.plot(valid_min_loss[0], valid_min_loss[1], color='black', marker='o',
             label=f'Validation Best Epoch - {valid_min_loss[0]}')
    plt.legend(loc='best')
    plt.title(label='Training & Validation Loss')
    plt.xlabel(xlabel='Epochs')
    plt.ylabel(ylabel='Loss')
    plt.show()

In [3]:
def accuracy_plot(hist):
  epochs = list(range(1, len(hist.history['accuracy']) + 1))

  plt.plot(epochs, hist.history['accuracy'], marker='o', linewidth=1, label='Training Accuracy', color='blue')
  plt.plot(epochs, hist.history['val_accuracy'], marker='o', linewidth=1, label='Validation Accuracy', color='red')
  plt.title(label='Training & Validation Accuracy')
  plt.legend(loc='best')
  plt.xlabel(xlabel='Epochs')
  plt.ylabel(ylabel='Accuracy')
  plt.show()

In [4]:
def get_model():
    model = keras.models.Sequential(layers=[
        Conv2D(filters=32, kernel_size=(3, 3), input_shape=(150, 150, 3),
            padding='same', activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dropout(rate=0.2),
        Dense(units=256, activation='relu'),
        Dense(units=4, activation='softmax')
    ])
    return model

DON'T RUN EVERY TIME, ONLY IF U WANT TO SHUFFLE TRAIN <-> TESTS

In [5]:
# import shutil
# output_dir = 'splitted_dataset'
# if os.path.exists(output_dir):
#    shutil.rmtree(output_dir, ignore_errors=True)

# os.makedirs(output_dir)
# import splitfolders
# splitfolders.ratio('dataset', output=output_dir, seed=1337, ratio=(.9, 0.1))

In [6]:
def load_images(path: str, class_id: int) -> list:
    content_list = os.listdir(path)
    files_list = [os.path.join(path, content_thing) for content_thing in content_list if os.path.isfile(os.path.join(path, content_thing))]
    imgs_list = []
    labels_list = []
    for file_path in files_list:
        temp_img = Image.open(file_path)
        numpy_image = np.asarray(temp_img.resize((150,150)))
        imgs_list.append(numpy_image)
        labels_list.append(class_id)
    return imgs_list, labels_list

# load train images
main_path = "splitted_dataset/"
cataract = main_path+"train/cataract"
diabetic_retinopathy = main_path+"train/diabetic_retinopathy"
glaucoma = main_path+"train/glaucoma"
normal = main_path+"train/normal"

In [7]:
images_cataract, labels_cataract = load_images(cataract, 1)
images_diabetic_retinopathy, labels_retinopathy = load_images(diabetic_retinopathy, 2)
images_glaucoma, labels_glaucoma = load_images(glaucoma, 3)
images_normal, labels_normal = load_images(normal, 0)

In [8]:
# merge tables
x = images_cataract+images_diabetic_retinopathy+images_glaucoma+images_normal
y = labels_cataract+labels_retinopathy+labels_glaucoma+labels_normal
x, y = shuffle_data(x, y)
del images_cataract, images_diabetic_retinopathy, images_glaucoma, images_normal, labels_cataract,labels_retinopathy,labels_glaucoma,labels_normal

In [9]:
images_cataract, labels_cataract = load_images(cataract, 1)
images_diabetic_retinopathy, labels_retinopathy = load_images(main_path+"val/diabetic_retinopathy", 2)
images_glaucoma, labels_glaucoma = load_images(main_path+"val/glaucoma", 3)
images_normal, labels_normal = load_images(main_path+"val/normal", 0)

In [10]:
X_test = images_cataract+images_diabetic_retinopathy+images_glaucoma+images_normal
Y_test = labels_cataract+labels_retinopathy+labels_glaucoma+labels_normal
X_test, Y_test = shuffle_data(X_test, Y_test)
del images_cataract, images_diabetic_retinopathy, images_glaucoma, images_normal, labels_cataract,labels_retinopathy,labels_glaucoma,labels_normal

In [11]:
train_data_gen = ToAugmantate()
test_data_gen = ToAugmantate(False)

In [12]:
kfolder = KFolder(5, x, y)

Computing folds


100%|██████████| 5/5 [00:01<00:00,  3.01it/s]


In [13]:
# # import tensorflow as tf
# # from dataset import get_data
# # from model import get_model
# # import os
# # import numpy as np
# # import matplotlib.pyplot as plt
# # import segmentation_models as sm
# # from folder import KFolder
# for i, fold in enumerate(kfolder.folds):
#     callbacks = [tf.keras.callbacks.ModelCheckpoint(f"./best_model_fold_{i}.h5", save_best_only=True, verbose=1)]
#     # callbacks = [tf.keras.callbacks.ModelCheckpoint("./best_model.h5", save_best_only=True, verbose=1)]
#     model = get_model()
#     model.compile(loss='mean_squared_logarithmic_error', optimizer='adam')

#     X_train, y_train, X_valid, y_valid = fold

#     model.fit(train_data_gen.flow(X_train, y=y_train, batch_size=8, subset='training'), validation_data=(train_data_gen.flow(X_valid, y_valid, batch_size=8, subset='validation')), batch_size=16, epochs=100, callbacks=callbacks, verbose=1)

#     del model
#     best_model = tf.keras.models.load_model(f"./best_model_fold_{i}.h5")

#     evaluation = best_model.evaluate(X_test, Y_test)

#     with open(f"./results_fold_{i}.txt",'w') as f:
#         f.write(f"test loss: {evaluation[0]}\n")
#         f.write(f"accuracy: {evaluation[1]}\n")
#         f.write(f"dice-score: {evaluation[2]}\n")
#     print(100*'--')


Below presented by Chat GPT, bc above was giving error I had no clue why

In [19]:
best_model = None
best_model_loss = float('inf')
best_model_fold_loss = []
best_model_accuracy = 0.0
best_model_fold_accuracy = []

for fold_idx, (x_train, y_train, x_valid, y_valid) in enumerate(kfolder.folds):
    print("Fold:", fold_idx + 1)
    model = get_model()
    optimizer = tf.keras.optimizers.Adam(0.0003)
    # Compile the model
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Define callbacks
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-7)

    # Train the model
    history = model.fit(train_data_gen.flow(x_train, y_train, batch_size=32),
                        epochs=35,
                        validation_data=test_data_gen.flow(x_valid, y_valid),
                        callbacks=[early_stopping, reduce_lr],
                        verbose=1)

    # Evaluate the model on validation data
    loss, accuracy = model.evaluate(test_data_gen.flow(x_valid, y_valid))
    print("Fold Loss:", loss)
    print("Fold Accuracy:", accuracy)

    # Save the best model for this fold
    if loss < best_model_loss:
        best_model = model
        best_model_loss = loss
    best_model_fold_loss.append(loss)
    best_model_fold_accuracy.append(accuracy)

# Print the best model's loss and accuracy for each fold
print("Best Model Loss for each fold:", best_model_fold_loss)
print("Best Model Accuracy for each fold:", best_model_fold_accuracy)

# Save the best overall model
best_model.save('best_model.h5')
print("Best Model saved as 'best_model.h5'")

Fold: 1
Epoch 1/35
95/95 [==============================] - 26s 263ms/step - loss: 2.1735 - categorical_accuracy: 0.3001 - val_loss: 0.7140 - val_categorical_accuracy: 0.3747 - lr: 3.0000e-04
Epoch 2/35
95/95 [==============================] - 23s 241ms/step - loss: 0.6521 - categorical_accuracy: 0.3011 - val_loss: 0.7997 - val_categorical_accuracy: 0.0013 - lr: 3.0000e-04
Epoch 3/35
95/95 [==============================] - 23s 247ms/step - loss: 0.5950 - categorical_accuracy: 0.2886 - val_loss: 0.5090 - val_categorical_accuracy: 0.1992 - lr: 3.0000e-04
Epoch 4/35
95/95 [==============================] - 23s 247ms/step - loss: 0.5118 - categorical_accuracy: 0.2876 - val_loss: 0.4537 - val_categorical_accuracy: 0.2177 - lr: 3.0000e-04
Epoch 5/35
95/95 [==============================] - 23s 240ms/step - loss: 0.5017 - categorical_accuracy: 0.2886 - val_loss: 0.4184 - val_categorical_accuracy: 0.2863 - lr: 3.0000e-04
Epoch 6/35
95/95 [==============================] - 23s 241ms/step - los